In [10]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score

from sklearn.svm import SVC

In [11]:
data = pd.read_csv('tatarstan_message_2.csv', sep = ';')
print(data.columns)
data = data[['categories', 'description_stem']]
data.head(3)

Index(['Unnamed: 0', 'categories', 'description', 'description_prep',
       'description_stem'],
      dtype='object')


,categories,description_stem
0,Благоустройство территории,трасса нижнекамск чистополь лесополоса пгт кам...
1,Благоустройство территории,двор дом ленинградский отсутствовать парковочн...
2,Содержание и ремонт муниципальных дорог,добрый день осуществляться уборка снег центр г...


LogisticRegression

In [12]:
#определяем модель "мешка слов"
data = data.dropna()
text_train = data['description_stem'].to_list()

vect = CountVectorizer(min_df=5).fit(text_train) #игнорировать слова, которые встречаются менее чем в min_df документах
#CountVectorizer по умолчанию сам производит токенизацию и выкидывает слова с длиной меньшей чем два.
print(vect.vocabulary_)

X_train = vect.transform(text_train)
y_train = data['categories'].to_list()

{'трасса': 4768, 'нижнекамск': 2481, 'чистополь': 5193, 'лесополоса': 1933, 'пгт': 2968, 'камский': 1581, 'поляна': 3311, 'проезжий': 3644, 'часть': 5164, 'опасно': 2715, 'нависать': 2237, 'накрениться': 2280, 'дерево': 949, 'снегопад': 4293, 'мочь': 2179, 'выдержать': 645, 'рухнуть': 4018, 'проходить': 3728, 'автотранспорт': 47, 'двор': 917, 'дом': 1021, 'ленинградский': 1922, 'отсутствовать': 2885, 'парковочный': 2954, 'место': 2091, 'житель': 1151, 'постоянно': 3397, 'мучиться': 2207, 'проезд': 3640, 'неоднократно': 2421, 'происходить': 3660, 'дтп': 1072, 'сторона': 4525, 'запретить': 1317, 'оставлять': 2779, 'машина': 2053, 'согласно': 4336, 'установленный': 4974, 'знак': 1410, 'предложение': 3457, 'расширение': 3922, 'авто': 23, 'фото': 5069, 'добрый': 996, 'день': 943, 'осуществляться': 2795, 'уборка': 4849, 'снег': 4292, 'центр': 5141, 'город': 824, 'гость': 839, 'воспользоваться': 561, 'платный': 3095, 'парковка': 2953, 'движение': 913, 'крайне': 1790, 'затруднить': 1363, 'выво

In [16]:
#определяем метрики для скоринга
scoring = {'Accuracy': make_scorer(accuracy_score), 
           'Precision': make_scorer(precision_score, average = 'weighted'), 
           'Recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')}

In [17]:
#определение наилучших параметров регуляризации (логистическая регрессия)
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)

C:\Anaconda\envs\python_37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Наилучшее значение перекрестной проверки: 0.80
Наилучшие параметры:  {'C': 1}


In [18]:
#кросс-валидация для "мешка слов" (логистическая регрессия)
scores = cross_validate(LogisticRegression(max_iter=10000, C = 1), X_train, y_train, scoring = scoring, cv=5)
print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")

C:\Anaconda\envs\python_37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels 

Accuracy перекр проверки: 0.8006068177761912
Precision перекр проверки: 0.8016630162088167
Recall перекр проверки: 0.8006068177761912
f1 перекр проверки: 0.7960243392956132


C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
#определяем модель "tf-idf"
data = data.dropna()
text_train = data['description_stem'].to_list()

vect = TfidfVectorizer(min_df=5).fit(text_train) #игнорировать слова, которые встречаются менее чем в min_df документах
print(vect.vocabulary_)

X_train = vect.transform(text_train)
y_train = data['categories'].to_list()

{'трасса': 4768, 'нижнекамск': 2481, 'чистополь': 5193, 'лесополоса': 1933, 'пгт': 2968, 'камский': 1581, 'поляна': 3311, 'проезжий': 3644, 'часть': 5164, 'опасно': 2715, 'нависать': 2237, 'накрениться': 2280, 'дерево': 949, 'снегопад': 4293, 'мочь': 2179, 'выдержать': 645, 'рухнуть': 4018, 'проходить': 3728, 'автотранспорт': 47, 'двор': 917, 'дом': 1021, 'ленинградский': 1922, 'отсутствовать': 2885, 'парковочный': 2954, 'место': 2091, 'житель': 1151, 'постоянно': 3397, 'мучиться': 2207, 'проезд': 3640, 'неоднократно': 2421, 'происходить': 3660, 'дтп': 1072, 'сторона': 4525, 'запретить': 1317, 'оставлять': 2779, 'машина': 2053, 'согласно': 4336, 'установленный': 4974, 'знак': 1410, 'предложение': 3457, 'расширение': 3922, 'авто': 23, 'фото': 5069, 'добрый': 996, 'день': 943, 'осуществляться': 2795, 'уборка': 4849, 'снег': 4292, 'центр': 5141, 'город': 824, 'гость': 839, 'воспользоваться': 561, 'платный': 3095, 'парковка': 2953, 'движение': 913, 'крайне': 1790, 'затруднить': 1363, 'выво

In [20]:
#определение наилучших параметров регуляризации (логистическая регрессия)
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)

C:\Anaconda\envs\python_37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Наилучшее значение перекрестной проверки: 0.81
Наилучшие параметры:  {'C': 10}


In [21]:
#кросс-валидация для "tf-idf" (логистическая регрессия)
scores = cross_validate(LogisticRegression(max_iter=10000, C = 10), X_train, y_train, scoring = scoring, cv=5)
print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")

C:\Anaconda\envs\python_37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels 

Accuracy перекр проверки: 0.8053185793325003
Precision перекр проверки: 0.8066960117864305
Recall перекр проверки: 0.8053185793325003
f1 перекр проверки: 0.8002917471947502


C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM

In [22]:
data = data.dropna()
text_train = data['description_stem'].to_list()
data = data.dropna()
text_train = data['description_stem'].to_list()

vect = TfidfVectorizer(min_df=5).fit(text_train) #игнорировать слова, которые встречаются менее чем в min_df документах

X_train = vect.transform(text_train)
y_train = data['categories'].to_list()
print(X_train)

  (0, 5193)	0.2419649834794235
  (0, 5164)	0.13253535196969443
  (0, 4768)	0.21706794930369874
  (0, 4293)	0.22015866481197535
  (0, 4018)	0.277977201952757
  (0, 3728)	0.17996058079023988
  (0, 3644)	0.142418126814633
  (0, 3311)	0.2756026837404905
  (0, 2968)	0.2756026837404905
  (0, 2715)	0.16957862872623766
  (0, 2481)	0.2226895588006144
  (0, 2280)	0.2675502344612076
  (0, 2237)	0.2713241599260953
  (0, 2179)	0.1343219024974561
  (0, 1933)	0.28975959407783153
  (0, 1581)	0.26581784862117136
  (0, 949)	0.14064050082168006
  (0, 645)	0.28975959407783153
  (0, 47)	0.21562764691728073
  (1, 5069)	0.1532994855817944
  (1, 4974)	0.18826985671331037
  (1, 4525)	0.11931791123495233
  (1, 4336)	0.18736883835772375
  (1, 3922)	0.2366874271524845
  (1, 3660)	0.19089966618622384
  :	:
  (28014, 940)	0.01758179887025656
  (28014, 917)	0.019628077414272714
  (28014, 913)	0.03810344352723242
  (28014, 905)	0.02678995289458987
  (28014, 903)	0.01154264848802535
  (28014, 835)	0.016112784794192724

In [23]:
#определение наилучших параметров регуляризации (SVM)
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(SVC(probability=False, kernel='rbf'), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)


C:\Anaconda\envs\python_37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Наилучшее значение перекрестной проверки: 0.81
Наилучшие параметры:  {'C': 10}


In [25]:
#кросс-валидация для "tf-idf" (SVM)
scores = cross_validate(SVC(probability=False, kernel='rbf', C = 10), X_train, y_train, scoring = scoring, cv=3)
print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")

C:\Anaconda\envs\python_37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
C:\Anaconda\envs\python_37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy перекр проверки: 0.801963507408488
Precision перекр проверки: 0.8020048804601069
Recall перекр проверки: 0.801963507408488
f1 перекр проверки: 0.7935485191387023
